In [199]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM

import time
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

In [102]:
#Create a path to where your data is stored.
path = r'C:\Users\Hiroshi\Documents\06-2024 ClimateWins'

In [103]:
#Read in the European weather data.
climate_unscaled = pd.read_csv(os.path.join(path, '01 Data','Weather_prediction.csv'))
climate_unscaled

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.0180,0.32,0.09,0,0.7,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1.0180,0.36,1.05,0,1.1,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,19600103,1,8,2.1,0.90,1.0180,0.18,0.30,0,0.0,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,19600104,1,3,2.1,0.92,1.0180,0.58,0.00,0,4.1,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,19600105,1,6,2.1,0.95,1.0180,0.65,0.14,0,5.4,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,20221027,10,1,2.1,0.79,1.0248,1.34,0.22,0,7.7,...,5,0.82,1.0142,1.13,0.41,0,3.4,10.7,7.9,13.5
22946,20221028,10,6,2.1,0.77,1.0244,1.34,0.22,0,5.4,...,5,0.82,1.0142,1.13,0.41,0,3.4,10.7,7.9,13.5
22947,20221029,10,4,2.1,0.76,1.0227,1.34,0.22,0,6.1,...,5,0.82,1.0142,1.13,0.41,0,3.4,10.7,7.9,13.5
22948,20221030,10,5,2.1,0.80,1.0212,1.34,0.22,0,5.8,...,5,0.82,1.0142,1.13,0.41,0,3.4,10.7,7.9,13.5


In [104]:
#Read in the pleasant weather data.
weather = pd.read_csv(os.path.join(path, '01 Data','Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
weather

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,20221027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22946,20221028,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22947,20221029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22948,20221030,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [105]:
#Drop the DATE, MONTH and stations with incomplete variables (Gdansk, Roma,Tours)
climate_nodate = climate_unscaled.drop(['DATE','MONTH',
                                       'GDANSK_cloud_cover',
                                        'GDANSK_humidity',
                                        'GDANSK_precipitation',
                                        'GDANSK_snow_depth',
                                        'GDANSK_temp_mean',
                                        'GDANSK_temp_min',
                                        'GDANSK_temp_max',
                                        'ROMA_cloud_cover',
                                        'ROMA_wind_speed',
                                        'ROMA_humidity','ROMA_pressure',
                                        'ROMA_sunshine',
                                        'ROMA_temp_mean',
                                        'TOURS_wind_speed',
                                        'TOURS_humidity',
                                        'TOURS_pressure',
                                        'TOURS_global_radiation',
                                        'TOURS_precipitation',
                                        'TOURS_temp_mean',
                                        'TOURS_temp_min',
                                        'TOURS_temp_max'], axis=1)

In [106]:
#Drop the snow depth variable
climate_nodate = climate_nodate.loc[:, ~climate_nodate.columns.str.endswith('_snow_depth')]

In [107]:
#Copy cloud cover data from Ljubljana to Kassel
position1 = climate_nodate.columns.get_loc('KASSEL_wind_speed')-1
climate_nodate.insert(position1,'KASSEL_cloud_cover',climate_nodate['LJUBLJANA_cloud_cover'])

In [108]:
#Copy wind speed and pressure data from Sonnblick to Munchen
position2 = climate_nodate.columns.get_loc('MUNCHENB_humidity')-1
position3 = climate_nodate.columns.get_loc('MUNCHENB_global_radiation')-1
climate_nodate.insert(position2,'MUNCHENB_wind_speed',climate_nodate['SONNBLICK_wind_speed'])
climate_nodate.insert(position3,'MUNCHENB_pressure',climate_nodate['SONNBLICK_pressure'])

In [109]:
#Copy wind speed and humidity data from Oslo to Stockholm
position4 = climate_nodate.columns.get_loc('STOCKHOLM_pressure')-1
climate_nodate.insert(position4,'STOCKHOLM_humidity',climate_nodate['OSLO_humidity'])
position5 = climate_nodate.columns.get_loc('STOCKHOLM_humidity')-1
climate_nodate.insert(position5,'STOCKHOLM_wind_speed',climate_nodate['OSLO_wind_speed'])

In [110]:
# Drop date column from prediction data
weather_nodate = weather.drop(['DATE'], axis=1)

In [111]:
weather_nodate.shape

(22950, 15)

In [112]:
climate_nodate.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [113]:
weather_nodate.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
#Drop the wind speed variable
climate_nodate = climate_nodate.loc[:, ~climate_nodate.columns.str.endswith('_wind_speed')]

In [115]:
climate_nodate.shape

(22950, 135)

In [116]:
climate_nodate.to_csv(os.path.join(path, '01 Data','climate_nodate_cleaned.csv'))

In [117]:
#Create an 'X' and 'y'matrix 
X = climate_nodate
y = weather_nodate
X

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.0180,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.0180,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.0180,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.0180,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.0180,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,1,0.79,1.0248,1.34,0.22,7.7,15.9,11.4,21.4,2,...,14.2,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22946,6,0.77,1.0244,1.34,0.22,5.4,16.7,14.3,21.9,0,...,14.3,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22947,4,0.76,1.0227,1.34,0.22,6.1,16.7,13.1,22.4,2,...,14.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22948,5,0.80,1.0212,1.34,0.22,5.8,15.4,11.6,21.1,1,...,12.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5


In [118]:
X = np.array(X)
y = np.array(y)

In [119]:
X = X.reshape(-1,15,9)

In [120]:
X.shape

(22950, 15, 9)

In [121]:
y.shape

(22950, 15)

In [122]:
#Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [123]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18360, 15, 9) (18360, 15)
(4590, 15, 9) (4590, 15)


In [124]:
epochs = 5
batch_size = 16
n_hidden = 2

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) #Don't use relu here!

C:\Users\Hiroshi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [125]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [126]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/5
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.0532 - loss: 8.6307 - val_accuracy: 0.0447 - val_loss: 8.6365
Epoch 2/5
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.0825 - loss: 8.8126 - val_accuracy: 0.0399 - val_loss: 8.6323
Epoch 3/5
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.0783 - loss: 8.6531 - val_accuracy: 0.0379 - val_loss: 8.6566
Epoch 4/5
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.0751 - loss: 8.6938 - val_accuracy: 0.0312 - val_loss: 8.7388
Epoch 5/5
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.0802 - loss: 8.8423 - val_accuracy: 0.0346 - val_loss: 8.7972


In [127]:
predictions = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [128]:
y_train.shape

(18360, 15)

In [129]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([predictions[y] for y in np.argmax(Y_true, axis=-1)])
    Y_pred = pd.Series([predictions[y] for y in np.argmax(Y_pred, axis=-1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [130]:
pred = model.predict(X_test)
pred.shape

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


(4590, 15)

In [131]:
y_test.shape

(4590, 15)

In [132]:
# Evaluate
print(confusion_matrix(y_test, pred))

Pred        BELGRADE  LJUBLJANA  MADRID
True                                   
BASEL              3       1703    1230
BELGRADE           1         91     810
BUDAPEST           1          6     153
DEBILT             1          0      68
DUSSELDORF         1          0      23
HEATHROW           2          9      70
KASSEL             0          0      11
LJUBLJANA          0          7      49
MAASTRICHT         0          2       4
MADRID             2        170     151
MUNCHENB           0          0       7
OSLO               0          1       7
STOCKHOLM          0          0       3
VALENTIA           0          1       3


In [133]:
epochs = 8
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) #Don't use relu here!

C:\Users\Hiroshi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [134]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [135]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.1116 - loss: 10.3977 - val_accuracy: 0.0505 - val_loss: 9.2894
Epoch 2/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.0946 - loss: 10.8576 - val_accuracy: 0.0407 - val_loss: 9.7412
Epoch 3/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.0932 - loss: 11.1038 - val_accuracy: 0.0277 - val_loss: 10.4135
Epoch 4/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.0913 - loss: 11.2678 - val_accuracy: 0.0373 - val_loss: 10.8719
Epoch 5/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.0876 - loss: 11.9540 - val_accuracy: 0.0366 - val_loss: 11.2739
Epoch 6/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.0885 - loss: 12.1782 - val_accuracy: 0.0440 - val_loss: 11.7119
Epoch 7/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.0822 - loss: 12.3272 - val_accuracy: 0.0390 - val_loss: 12.0356
Epoch 8/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.0849 - loss: 12.36

In [136]:
weather_nodate.dtypes

BASEL_pleasant_weather         int64
BELGRADE_pleasant_weather      int64
BUDAPEST_pleasant_weather      int64
DEBILT_pleasant_weather        int64
DUSSELDORF_pleasant_weather    int64
HEATHROW_pleasant_weather      int64
KASSEL_pleasant_weather        int64
LJUBLJANA_pleasant_weather     int64
MAASTRICHT_pleasant_weather    int64
MADRID_pleasant_weather        int64
MUNCHENB_pleasant_weather      int64
OSLO_pleasant_weather          int64
SONNBLICK_pleasant_weather     int64
STOCKHOLM_pleasant_weather     int64
VALENTIA_pleasant_weather      int64
dtype: object

In [137]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([predictions[y] for y in np.argmax(Y_true, axis=-1)])
    Y_pred = pd.Series([predictions[y] for y in np.argmax(Y_pred, axis=-1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [138]:
pred = model.predict(X_test)
pred.shape

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


(4590, 15)

In [139]:
y_test.shape

(4590, 15)

In [140]:
# Evaluate
print(confusion_matrix(y_test, pred))

Pred        BASEL  BELGRADE  MADRID
True                               
BASEL        2935         1       0
BELGRADE      900         0       2
BUDAPEST      159         0       1
DEBILT         69         0       0
DUSSELDORF     24         0       0
HEATHROW       80         0       1
KASSEL         11         0       0
LJUBLJANA      54         0       2
MAASTRICHT      6         0       0
MADRID        320         0       3
MUNCHENB        7         0       0
OSLO            8         0       0
STOCKHOLM       3         0       0
VALENTIA        4         0       0


#### CNN

In [142]:
epochs = 8
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh'))

C:\Users\Hiroshi\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [143]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [144]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.1529 - loss: 26.0681 - val_accuracy: 0.1806 - val_loss: 18.1197
Epoch 2/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.1764 - loss: 17.3859 - val_accuracy: 0.1869 - val_loss: 18.0319
Epoch 3/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.1789 - loss: 17.2332 - val_accuracy: 0.1865 - val_loss: 18.0284
Epoch 4/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.1787 - loss: 17.7030 - val_accuracy: 0.1863 - val_loss: 18.0143
Epoch 5/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.1797 - loss: 17.4138 - val_accuracy: 0.1867 - val_loss: 18.0073
Epoch 6/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1814 - loss: 17.7592 - val_accuracy: 0.1869 - val_loss: 18.0073
Epoch 7/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1797 - loss: 17.5670 - val_accuracy: 0.1867 - val_loss: 18.0073
Epoch 8/8
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.1778 - loss: 17.

In [145]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([predictions[y] for y in np.argmax(Y_true, axis=-1)])
    Y_pred = pd.Series([predictions[y] for y in np.argmax(Y_pred, axis=-1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [146]:
pred = model.predict(X_test)
pred.shape

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


(4590, 15)

In [147]:
# Evaluate
print(confusion_matrix(y_test, pred))

Pred        BASEL  BELGRADE  BUDAPEST  KASSEL  MAASTRICHT  MADRID  MUNCHENB
True                                                                       
BASEL         786        27       110     312         188    1153       360
BELGRADE      502         7        71      58          80      70       114
BUDAPEST       75         4        18      16          17      11        19
DEBILT         33         5        10       7           6       0         8
DUSSELDORF     13         1         3       2           0       0         5
HEATHROW       37         4         3      20           4       2        11
KASSEL          3         0         2       2           2       0         2
LJUBLJANA      15         0         1       9           7       9        15
MAASTRICHT      1         0         0       0           0       1         4
MADRID        101         0         2      83          12      39        86
MUNCHENB        1         0         1       0           0       2         3
OSLO        

#### Task 2.4